<a href="https://colab.research.google.com/github/njucs/notebook/blob/master/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **单句，预测下一个词**

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

# 准备数据
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)

word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx) # 预测下一个词其实就是个分类问题，类别数就是词表大小，此处为19
max_len = len(sentence.split()) # 最长的句子长度，此处为21
n_hidden = 5

# 数据预处理，构建 dataset，定义 dataloader
# input 的长度永远保持 max_len，并且循环了 max_len-1 次
# 所以最终 input_batch 的维度是 [max_len - 1, max_len, n_class]
# 此处是以一句话为例，如果是一个数据集，增加一个循环将每一句话同样处理放入batch中
def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i in range(max_len - 1):
        input = [word2idx[n] for n in words[:(i + 1)]] # a[:x]表示取前x个数据（不含x）
        input = input + [0] * (max_len - len(input)) # 后面用idx 0来补齐输入，如当i=0，Github的idx是10时，此处为[10, 0, 0, ..., 0]，长度为max_len
        target = word2idx[words[i + 1]] # target是下一个词（待预测的词）
        input_batch.append(np.eye(n_class)[input]) # 根据input中index序列生成one-hot形式数组，每个词一个ont-hot表示，循环max_len - 1次后，最终的维度是torch.Size([20, 21, 19])
        target_batch.append(target) # torch.Size([20])

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

# input_batch: [max_len - 1, max_len, n_class]
input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch) # 包装数据和目标张量的数据集。通过每一个 tensor 的第一个维度进行索引，因此，该类中的 tensor 第一维度必须相等
loader = Data.DataLoader(dataset, 16, True) # batch size = 16，每次迭代训练时将数据洗牌（shuffle is True）

class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        # 全连接层做分类
        # 接BiLSTM，故n_hidden * 2
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # input : [max_len, batch_size, n_class]

        # 按照LSTM网络的初始化要求，随机初始化 hidden_state 和 cell_state
        hidden_state = torch.randn(1*2, batch_size, n_hidden)   # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden)     # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        # h 和 c 本例子不需要使用，故直接忽略
        # outputs = [seq_len (max_len), batch_size, hidden_size * num_directions]
        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # 只需要取最后一个输出即可 [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model

model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(10000):
    for x, y in loader:
      pred = model(x)
      loss = criterion(pred, y)
      if (epoch + 1) % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

# Pred
predict = model(input_batch).data.max(1, keepdim=True)[1] # 按维度 1 返回最大值，并且返回索引，keepdim = True 表示输出和输入的维度一样（不会压缩维度）
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 1.941703
Epoch: 1000 cost = 2.065655
Epoch: 2000 cost = 1.316938
Epoch: 2000 cost = 1.289267
Epoch: 3000 cost = 0.939728
Epoch: 3000 cost = 0.851929
Epoch: 4000 cost = 0.771056
Epoch: 4000 cost = 0.502699
Epoch: 5000 cost = 0.536494
Epoch: 5000 cost = 0.576204
Epoch: 6000 cost = 0.496226
Epoch: 6000 cost = 0.286410
Epoch: 7000 cost = 0.393373
Epoch: 7000 cost = 0.422419
Epoch: 8000 cost = 0.340095
Epoch: 8000 cost = 0.974846
Epoch: 9000 cost = 0.367620
Epoch: 9000 cost = 0.556251
Epoch: 10000 cost = 0.344123
Epoch: 10000 cost = 0.136877
GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more
['makes', 'makes', 'it', 'easy', 'to', 'automate', 'all', 'your', 'workflows', 'workflows', 'from', 'continuous', 'integration', 'and', 'delivery', 'to', 'issue', 'triage', 'and', 'more']


### **句子集，预测下一个词**

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

# 准备数据
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)

word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx) # 预测下一个词其实就是个分类问题，类别数就是词表大小，此处为19
max_len = len(sentence.split()) # 最长的句子长度，此处为21
n_hidden = 5

# 数据预处理，构建 dataset，定义 dataloader
# input 的长度永远保持 max_len，并且循环了 max_len-1 次
# 所以最终 input_batch 的维度是 [max_len - 1, max_len, n_class]
# 此处是以一句话为例，如果是一个数据集，增加一个循环将每一句话同样处理放入batch中
def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i in range(max_len - 1):
        input = [word2idx[n] for n in words[:(i + 1)]] # a[:x]表示取前x个数据（不含x）
        input = input + [0] * (max_len - len(input)) # 后面用idx 0来补齐输入，如当i=0，Github的idx是10时，此处为[10, 0, 0, ..., 0]，长度为max_len
        target = word2idx[words[i + 1]] # target是下一个词（待预测的词）
        input_batch.append(np.eye(n_class)[input]) # 根据input中index序列生成one-hot形式数组，每个词一个ont-hot表示，循环max_len - 1次后，最终的维度是torch.Size([20, 21, 19])
        target_batch.append(target) # torch.Size([20])

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

# input_batch: [max_len - 1, max_len, n_class]
input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch) # 包装数据和目标张量的数据集。通过每一个 tensor 的第一个维度进行索引，因此，该类中的 tensor 第一维度必须相等
loader = Data.DataLoader(dataset, 16, True) # batch size = 16，每次迭代训练时将数据洗牌（shuffle is True）

class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        # 全连接层做分类
        # 接BiLSTM，故n_hidden * 2
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # input : [max_len, batch_size, n_class]

        # 按照LSTM网络的初始化要求，随机初始化 hidden_state 和 cell_state
        hidden_state = torch.randn(1*2, batch_size, n_hidden)   # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden)     # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        # h 和 c 本例子不需要使用，故直接忽略
        # outputs = [seq_len (max_len), batch_size, hidden_size * num_directions]
        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # 只需要取最后一个输出即可 [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model

model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(10000):
    for x, y in loader:
      pred = model(x)
      loss = criterion(pred, y)
      if (epoch + 1) % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

# Pred
predict = model(input_batch).data.max(1, keepdim=True)[1] # 按维度 1 返回最大值，并且返回索引，keepdim = True 表示输出和输入的维度一样（不会压缩维度）
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])